In [1]:
import cv2
import math
import numpy
import sys
#import pdb

#coordinate width and length
rows=520
cols=720

data_of_txt=[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
pre_pt=[0.0,0.0]
end_pt=pre_pt
pre_pt_1=pre_pt
end_pt_1=pre_pt
show_video_flag="true"
save_video_flag="true"
video_rate=12
hog_flag="true"
fixed_window_flag="false"
multiscale_flag="true"
lab_flag="true"
widthHeightRatio=1.0
#tmp_3=dst_3=src_3=numpy.zeros((rows,cols,3),numpy.uint8)

VIDEO_WINDOW_NAME="kcftrack result"
DISTANCE="distance"
video_file_name="result.avi"

In [2]:
def drawxy(image):
    #draw the coordinate
    cv2.line(image,(50,rows-50),(cols-50,rows-50),(0,255,255),1,8)
    cv2.line(image,(50,50),(50,rows-50),(0,255,255),1,8)
    cv2.line(image,(cols-80,rows-80),(cols-50,rows-50),(0,255,255),1,8)
    cv2.line(image,(cols-80,rows-20),(cols-50,rows-50),(0,255,255),1,8)
    cv2.line(image,(80,80),(50,50),(0,255,255),1,8)
    cv2.line(image,(20,80),(50,50),(0,255,255),1,8)
    
#    pdb.set_trace()
    #add annotation of coordinate
    font=cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(image,'time',(cols-120,rows-20),font,0.5,(0,255,255),1,8,0)
    cv2.putText(image,'distance',(30,40),font,0.5,(0,255,255),1,8,0)

    #show the coordinate
    cv2.imshow('result',image)
    #key=cv2.waitKey(0)


In [3]:
def readtxt():
    #get the data of following point
    lines=0
    data=open("data.txt")
    data_all_line=data.readlines()
    for data_line in data_all_line:
        data_of_txt[lines]=float(data_line.strip('\n'))
        lines=lines+1
    pre_pt=(data_of_txt[0],data_of_txt[1])
    end_pt=(data_of_txt[0]+data_of_txt[2],data_of_txt[1]+data_of_txt[3])
    pre_pt_1=(data_of_txt[4],data_of_txt[5])
    end_pt_1=(data_of_txt[4]+data_of_txt[6],data_of_txt[5]+data_of_txt[7])



In [4]:
input_video=cv2.VideoCapture("test.mp4")
#dis_3=cv2.imread("1.png")
tracker_types = ['BOOSTING', 'MIL','KCF', 'TLD', 'MEDIANFLOW', 'GOTURN', 'MOSSE', 'CSRT']

if input_video.isOpened()=='flase':
    print("video open failed")
    sys.exit()
#if show_video_flag!=0:
#cv2.namedWindow(VIDEO_WINDOW_NAME)
#cv2.namedWindow(DISTANCE)

In [5]:
#if lab_flag=='true':
#    hog_flag='ture'
tracker=cv2.TrackerKCF_create()
tracker1=cv2.TrackerKCF_create()
#kcf_init_end_flag='false'
#drawxy(dis_3)


In [6]:
ret='True'
#frame=1

In [7]:
width = input_video.get(cv2.CAP_PROP_FRAME_WIDTH)   
height = input_video.get(cv2.CAP_PROP_FRAME_HEIGHT) 
fourcc = cv2.VideoWriter_fourcc(*'DIVX')
width    
#if save_video_flag is True:
out = cv2.VideoWriter('./output.mp4',fourcc, 20.0, (int(width), int(height)))
        
ret,frame=input_video.read()
if ret is False:
    print("video end")
    sys.exit() 
    
readtxt()

#get the point of person and bike
init_xMin=pre_pt[0]
init_yMin=pre_pt[1]
init_xMax=end_pt[0]
init_yMax=end_pt[1]
init_width=init_xMax-init_xMin
init_height=init_yMax-init_yMin
ret1 = tracker.init(frame,(init_xMin,init_yMin,init_width,init_height))
init_xMin_1 =  pre_pt_1[0]
init_yMin_1 =  pre_pt_1[1]
init_xMax_1 =  end_pt_1[0]
init_yMax_1 =  end_pt_1[1]
init_width_1 = init_xMax_1 - init_xMin_1
init_height_1 = init_yMax_1 - init_yMin_1
ret = tracker1.init(frame,(init_xMin_1,init_yMin_1,init_width_1,init_height_1)) & ret1

In [9]:
while ret:
    ret,frame=input_video.read()
    ret
    if not ret:
        break
        
    #start timer
    timer=cv2.getTickCount()
    
    #update tracker
    ret,bbox=tracker.update(frame)
    ret1,bbox1=tracker1.update(frame)
    
    #calculate frames per second
    fps=cv2.getTickFrequency()/(cv2.getTickCount()-timer)
    
    if ret&ret1:
        p1 = (int(bbox[0]), int(bbox[1]))
        p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
        new_frame = cv2.rectangle(frame, p1, p2, (255,0,0), 2, 1)
        
        p3 = (int(bbox1[0]), int(bbox1[1]))
        p3 = (int(bbox1[0] + bbox1[2]), int(bbox1[1] + bbox1[3]))
        new_frame = cv2.rectangle(new_frame, p3, p4, (255,0,0), 2, 1)
    
    else :
        cv2.putText(new_frame,"Tracking failure detected",(100,80),cv2.FONT_HERSHEY_SIMPLEX,0.75,(0,0,255),2)
    
    cv2.putText(new_frame, tracker_types[2] + " Tracker", (100,20), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50),2);
     
        # Display FPS on frame
    cv2.putText(new_frame, "FPS : " + str(int(fps)), (100,50), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50), 2);
 
        # Display result
    out.write(new_frame)
    cv2.imshow("Tracking", new_frame)
        
out.release()
input_video.release()
cv2.destroyAllWindows()
        #get the picture of this frame
